In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from scipy.ndimage.measurements import label
import os 
import pickle 
import time
from utils.utils import *
from moviepy.editor import *

%matplotlib inline

## Load pretrained classifier

In [2]:
clf, X_scaler, pca = pickle.load( open( "trained_classifier.p", "rb" ) )

## Parameters

In [3]:
color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 16 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
spatial_size = (16, 16) # Spatial binning dimensions
hist_bins = 16    # Number of histogram bins
spatial_feat = False # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

## Define class to keep previous detections

In [4]:
class Detections():
    def __init__(self, k):
        self.prev_windows = []   # History of detections
        self.k = k             # Number of frames to consider
        for i in range(self.k):
            self.prev_windows.append(None)
        
    def add_detections(self, windows):
        self.prev_windows.append(windows)
        self.prev_windows = self.prev_windows[1:]
        
    def get_detections(self):
        return [win for win in self.prev_windows if win is not None]
        
    def get_num_stored_frames(self):
        return sum(x is not None for x in self.prev_windows)

In [5]:
def process_frame(img):
    rectangles = []
    
    threshold = 2
    color_space = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    orient = 9  # HOG orientations
    pix_per_cell = 16 # HOG pixels per cell
    cell_per_block = 2 # HOG cells per block
    hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"
    
    # Get hot windows
    scales = [1,1,1.5,1.5,2,2,3,3]
    y_starts = [400,416,400,432,400,432,400,464]
    y_stops = [464,480,496,528,528,560,596,660]
    rects = []
    for i, scale in enumerate(scales):
        ystart = y_starts[i]
        ystop = y_stops[i]
        rects.append(find_cars(img, ystart, ystop, scale, clf, X_scaler, pca, orient, pix_per_cell, cell_per_block, 
                               hist_bins, color_space))

    windows = [item for sublist in rects for item in sublist] 
    
    if len(windows) > 0:
        history.add_detections(windows)
    
    heatmap_img = np.zeros_like(img[:,:,0])
    for win in history.get_detections():
        heatmap_img = add_heat(heatmap_img, win)
        
    heatmap_img = apply_threshold(heatmap_img, 1 + history.get_num_stored_frames()//2)
    labels = label(heatmap_img)
    draw_img, rects = draw_labeled_bboxes(np.copy(img), labels, color=(0,255,0), thickness = 8)
    return draw_img

In [6]:
history = Detections(15)

test_out_file = './output_videos/test_video_out.mp4'
clip_test = VideoFileClip('test_video.mp4')
clip_test_out = clip_test.fl_image(process_frame)
%time clip_test_out.write_videofile(test_out_file, audio=False)

[MoviePy] >>>> Building video ./output_videos/test_video_out.mp4
[MoviePy] Writing video ./output_videos/test_video_out.mp4


 97%|█████████▋| 38/39 [00:28<00:00,  1.47it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/test_video_out.mp4 

CPU times: user 2min 20s, sys: 4.64 s, total: 2min 25s
Wall time: 29.9 s


In [ ]:
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(test_out_file))

In [10]:
history = Detections(10)

project_out_file = './output_videos/project_video_out.mp4'
clip_project = VideoFileClip('project_video.mp4')
clip_project_out = clip_project.fl_image(process_frame)
%time clip_project_out.write_videofile(project_out_file, audio=False)

[MoviePy] >>>> Building video ./output_videos/project_video_out.mp4
[MoviePy] Writing video ./output_videos/project_video_out.mp4


100%|█████████▉| 1260/1261 [16:42<00:00,  1.36it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./output_videos/project_video_out.mp4 

CPU times: user 1h 18min 23s, sys: 2min 43s, total: 1h 21min 7s
Wall time: 16min 43s


In [ ]:
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_out_file))